In [10]:
import numpy as np
import torch
import pandas as pd

In [9]:
import os
os.getcwd()

'/tmp/rb4XCe7OTU'

# Testing

In [11]:
test_results = pd.read_csv("/data/users/etosato/ANM_Verona/results/runs/all_testing_results.csv")
test_results["run_id"] = test_results["run_id"].astype(str).str.replace("run", "", regex=False).astype(int)
display(test_results)

,run_id,group,seed,accuracy,precision,recall,f1
0,1,ADNI vs PSP,42,0.519,0.471,0.667,0.552
1,2,ADNI vs PSP,123,0.593,0.571,0.333,0.421
2,3,ADNI vs PSP,2023,0.519,0.444,0.333,0.381
3,4,ADNI vs PSP,31415,0.444,0.364,0.333,0.348
4,5,ADNI vs PSP,98765,0.296,0.111,0.083,0.095
5,6,ADNI vs CBS,42,0.583,0.444,0.444,0.444
6,7,ADNI vs CBS,123,0.625,0.500,0.778,0.609
7,8,ADNI vs CBS,2023,0.708,1.000,0.222,0.364
8,9,ADNI vs CBS,31415,0.458,0.357,0.556,0.435
9,10,ADNI vs CBS,98765,0.625,0.500,0.444,0.471


In [15]:
metrics_cols = ['accuracy', 'precision', 'recall', 'f1']
group_stats = test_results.groupby("group")[metrics_cols].agg(['mean', 'std'])
display(round(group_stats,3))
print("\n",round(group_stats,3))

accuracy        precision        recall            f1       
                mean    std      mean    std   mean    std   mean    std
group                                                                   
ADNI vs CBS    0.600  0.091     0.560  0.253  0.489  0.202  0.465  0.090
ADNI vs PSP    0.474  0.113     0.392  0.174  0.350  0.208  0.359  0.167
PSP vs CBS     0.562  0.052     0.589  0.037  0.784  0.126  0.669  0.052


             accuracy        precision        recall            f1       
                mean    std      mean    std   mean    std   mean    std
group                                                                   
ADNI vs CBS    0.600  0.091     0.560  0.253  0.489  0.202  0.465  0.090
ADNI vs PSP    0.474  0.113     0.392  0.174  0.350  0.208  0.359  0.167
PSP vs CBS     0.562  0.052     0.589  0.037  0.784  0.126  0.669  0.052


# Example

In [3]:
volume = np.load("/data/users/etosato/ANM_Verona/data/FCmaps_processed/1_S_5000.processed.npy")
print("Shape of the volume:", volume.shape)

Shape of the volume: (91, 109, 91)


# Checkpoints

Example of checkpoint loading

In [7]:
# Path to the saved checkpoint
ckpt_path = "/cnn/runs/run1/best_model_fold2.pt"

# Load checkpoint
checkpoint = torch.load(ckpt_path, map_location="cpu", weights_only=False)

# Show all available keys in the checkpoint
print("Available keys in the checkpoint:")
print(checkpoint.keys())

Available keys in the checkpoint:
dict_keys(['state_dict', 'optimizer_state', 'val_accuracy', 'epoch', 'best_train_loss', 'best_val_loss', 'fold'])


In [8]:
# Training information
epoch = checkpoint["epoch"]
val_accuracy = checkpoint["val_accuracy"]
train_loss = checkpoint["best_train_loss"]
val_loss = checkpoint["best_val_loss"]
fold = checkpoint["fold"]

print(f"Fold: {fold}")
print(f"Best Epoch: {epoch}")
print(f"Validation Accuracy: {val_accuracy:.3f}")
print(f"Best Training Loss: {train_loss:.3f}")
print(f"Best Validation Loss: {val_loss:.3f}")


Fold: 2
Best Epoch: 10
Validation Accuracy: 0.714
Best Training Loss: 0.560
Best Validation Loss: 0.971


# Tuning

In [22]:
# Load the CSV file
csv_path = "/src/cnn/tuning/adni_psp.csv"
df = pd.read_csv(csv_path)

# Sort: highest best_accuracy first, then lowest avg_val_loss
sorted_df_acc = df.sort_values(by=["best_accuracy", "avg_val_loss"], ascending=[False, True])

# Select the best configuration
best_config_acc = sorted_df_acc.iloc[0]

print("Best configuration:")
print(best_config_acc)

Best configuration:
tuning                      2
config                config8
group             ADNI vs PSP
threshold              no thr
best_fold                   3
best_accuracy           0.905
avg_accuracy            0.819
avg_train_loss           0.16
avg_val_loss            0.708
optimizer                adam
batch_size                  8
lr                     0.0001
weight_decay            0.001
model_type           densenet
epochs                     50
test size                 0.2
Name: 7, dtype: object


In [21]:
top_10_configs = sorted_df_acc[[
    "tuning", "config", "best_accuracy", "avg_accuracy", "avg_val_loss",
    "optimizer", "batch_size", "lr", "weight_decay", "model_type"
]].head(10)

print("Top 10 configurations (by best_accuracy, then avg_val_loss):")
display(top_10_configs)


Top 10 configurations (by best_accuracy, then avg_val_loss):


,tuning,config,best_accuracy,avg_accuracy,avg_val_loss,optimizer,batch_size,lr,weight_decay,model_type
7,2,config8,0.905,0.819,0.708,adam,8,0.0001,0.0010,densenet
4,2,config5,0.905,0.800,0.970,adam,8,0.0001,0.0001,resnet
6,2,config7,0.905,0.810,1.016,adam,8,0.0001,0.0010,resnet
16,6,config1,0.857,0.657,0.615,adam,4,0.0010,0.0010,resnet
17,6,config2,0.857,0.810,0.641,adam,4,0.0010,0.0010,densenet
0,2,config1,0.857,0.810,0.675,adam,8,0.0010,0.0001,resnet
15,2,config16,0.857,0.800,0.680,sgd,8,0.0001,0.0010,densenet
13,2,config14,0.857,0.810,0.697,sgd,8,0.0001,0.0001,densenet
20,6,config5,0.857,0.829,0.713,adam,4,0.0000,0.0010,densenet
14,2,config15,0.857,0.771,0.744,sgd,8,0.0001,0.0010,resnet


In [11]:
sorted_df_loss = df.sort_values(by=["avg_val_loss", "best_accuracy"], ascending=[True, False])

# Select the best configuration
best_config_loss = sorted_df_loss.iloc[0]

print("Best configuration:")
print(best_config_loss)

Best configuration:
tuning                      6
config                config1
group             ADNI vs PSP
threshold              no thr
best_fold                   4
best_accuracy           0.857
avg_accuracy            0.657
avg_train_loss          0.606
avg_val_loss            0.615
optimizer                adam
batch_size                  4
lr                      0.001
weight_decay            0.001
model_type             resnet
epochs                     50
test size                 0.2
Name: 16, dtype: object


In [ ]:
sorted_df_loss